![OER Header](img/header.png)

# Synthetic Biomedical Data – Lesson 3: Irrelevant Features

Part of the *Microcredit Synthetic Biomedical Data*.

➡️ [Back to Lesson 3a: Irrelevant Features (Noise)](03a_irrelevant_features_noise.ipynb)
➡️ [Module Overview (README)](../README.md)

---

## Recap from Lesson 3a
You:
- Generated synthetic biomedical datasets with informative and noise features.
- Visualized feature distributions and class separability.
- Evaluated cohen's d effect size for informative features and noise features.
- Understood how effect size ranking is influenced by feature dimension and sample size.

Now we focus on the effect of the dimension and distribution of **irrelevant features** — variables that carry *no* useful information about the label.


If too many irrelevant features are present, they can:
- Make it harder to identify true biomarkers. Too many irrelevant features can hide the true signal.
- Increase model complexity and training time and make feature selection more difficult.
- Lead to overfitting if models start learning patterns in the noise.

Synthetic data allows us to deliberately add irrelevant features to test how models and methods cope with them.

## What you'll learn
In this lesson, we will add irrelevant features to a synthetic dataset
and see how they effect model performance and feature correlations. After completing this notebook, you will be able to:
- Add purely irrelevant features with configurable distributions (normal, uniform, Laplace).
- Visualize the distributions of irrelevant features.
- Evaluate how increasing the number of irrelevant features affects model performance using cross-validated balanced accuracy.
- Analyze spurious correlations between irrelevant features and the label.

### Code – Imports, Installation/Upgrade

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from biomedical_data_generator import ClassConfig, DatasetConfig, generate_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, make_scorer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler

# Configure plotting for better readability
sns.set_theme(style="whitegrid", context="notebook")

## Data configuration from irrelevant features lesson

In [ ]:
n_noise_features = 1200  # number of irrelevant features to add
n_class_samples = 15  # samples per class, analogous to a small biomedical study
cfg_noise_normal = DatasetConfig(
    n_informative=8,
    n_noise=n_noise_features,
    class_configs=[
        ClassConfig(n_samples=n_class_samples),  # two balanced classes
        ClassConfig(n_samples=n_class_samples),
    ],
    class_sep=[1.2],  # keep consistent with the baseline
    prefixed_feature_naming=True,  # i1..i8, n1..n12
    noise_distribution="normal",
    random_state=1908,
)
xn, yn, meta_n = generate_dataset(cfg_noise_normal, return_dataframe=True)

# Combine X and y for plotting
x_noise = xn.copy()
x_noise.insert(0, "class", yn)

# Step 1 - Inspect noise distributions
## Normal noise
With `noise_distribution="normal"` and `noise_scale=1.0`, we expect:
- Approximately zero mean and unit variance for noise columns.
- No visible class separation in the noise columns.
- Similar spread across noise columns, up to sampling variability.


In [ ]:
sns.histplot(data=x_noise, x="n42", hue="class", kde=True)

In [ ]:
print("Comparison of informative features alone versus the same features with additional noise features:")
# informative features only
X_tr1, X_te1, y_tr1, y_te1 = train_test_split(xn.iloc[:, :8].values, yn, test_size=0.3, random_state=42, stratify=yn)
clf = make_pipeline(StandardScaler(), LogisticRegression(max_iter=500, random_state=42))
clf.fit(X_tr1, y_tr1)

pred = clf.predict(X_te1)
print("Logistic Regression balanced accuracy (informative only):", round(balanced_accuracy_score(y_te1, pred), 3))

# added 1200 noise features
X_tr, X_te, y_tr, y_te = train_test_split(xn.values, yn, test_size=0.3, random_state=42, stratify=yn)

clf = make_pipeline(StandardScaler(), LogisticRegression(max_iter=500, random_state=42))
clf.fit(X_tr, y_tr)
pred = clf.predict(X_te)
print(
    "Logistic Regression balanced accuracy (with added noise features):",
    round(balanced_accuracy_score(y_te, pred), 3),
)

## Uniform noise
Now we switch to `noise_distribution="uniform"`.
Here, low and high set the bounds of the interval, so values are uniformly distributed in `[low, high]`. We use noise_distribution_params={"low": -2, "high": 2}, so noise features will mostly lie in `[-2, 2]` with a flat (uniform) density.

In [ ]:
cfg_noise_uniform = DatasetConfig(
    n_informative=8,
    n_noise=1200,
    class_configs=[
        ClassConfig(n_samples=5000),  # two balanced classes
        ClassConfig(n_samples=5000),  # set a larger sample size for better empirical estimates
    ],
    noise_distribution="uniform",
    noise_distribution_params={"low": -2, "high": 2},
    random_state=42,
)
xu, yu, meta_u = generate_dataset(cfg_noise_uniform, return_dataframe=True)
xu.insert(0, "class", yu)

# Empirical min/max of first 5 noise columns to verify ~[-2, 2]
mins = xu[[f"n{k}" for k in range(1, 6)]].min().round(3)
maxs = xu[[f"n{k}" for k in range(1, 6)]].max().round(3)
display(pd.DataFrame({"min": mins, "max": maxs}))

In [ ]:
sns.histplot(data=xu, x="n2", hue="class")

### Uniform noise expectations

With `noise_distribution="uniform"` and `noise_scale=2.0`, values are roughly in **`[-2, 2]`**.
You should observe:
- Empirical **min/max** close to −2 and +2 (but not equal—finite samples fall short of true bounds).
- **Nearly constant density** within the interval, unlike the bell shape of normal noise.

# Step 2 - The influence of noise on model performance
## Quick train/test split on the normal-noise dataset to see if noise hurts performance

In [ ]:
print("Comparison of informative features alone versus the same features with additional noise features:")
# informative features only
X_tr1, X_te1, y_tr1, y_te1 = train_test_split(xn.iloc[:, :8].values, yn, test_size=0.3, random_state=42, stratify=yn)
clf = make_pipeline(StandardScaler(), LogisticRegression(max_iter=500, random_state=42))
clf.fit(X_tr1, y_tr1)

pred = clf.predict(X_te1)
print("Logistic Regression balanced accuracy (informative only):", round(balanced_accuracy_score(y_te1, pred), 3))

# added 1200 noise features
X_tr, X_te, y_tr, y_te = train_test_split(xn.values, yn, test_size=0.3, random_state=42, stratify=yn)

clf = make_pipeline(StandardScaler(), LogisticRegression(max_iter=500, random_state=42))
clf.fit(X_tr, y_tr)
pred = clf.predict(X_te)
print(
    "Logistic Regression balanced accuracy (8 informative with 1200 noise features added):",
    round(balanced_accuracy_score(y_te, pred), 3),
)

# Step 3 — Balanced accuracy vs. number of noise features (cross-validated)

We now study how *increasing the number of irrelevant (noise) features* affects model performance. We measure how a simple linear classifier (Logistic Regression) behaves under **Stratified 5-Fold CV** as we increase the number of standard normal distributed noise features.

**Plan**
1. Keep the true signal fixed: 8 informative features, same effect size/separability.
2. Vary the number of **pure noise** features: e.g. `k ∈ {0, 50, 200, 1000}`.
3. Train a simple linear classifier (Logistic Regression) **with scaling** and evaluate via **Stratified 5-Fold CV**. We keep the model fixed to isolate the effect of dimensionality.
4. Only one factor changes: the number of irrelevant features.
5. Track: **Balanced accuracy** (robust to small class imbalances)

In [ ]:
# Experiment configuration

# Noise counts to sweep over
NOISE_COUNTS = [0, 50, 200, 1000, 10000]

# Estimator: scale -> logistic regression (binary)
# liblinear is robust for small-sample, L2 by default
ESTIMATOR = make_pipeline(RobustScaler(), LogisticRegression(solver="liblinear", random_state=42, max_iter=2000))

CV = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
SCORER = make_scorer(balanced_accuracy_score)

### Why logistic regression + scaling?

- **Logistic Regression** is a transparent linear baseline; with L2 regularization it copes reasonably with moderate noise.
- **RobustScaler** avoids feature-scale dominance and reduces optimization issues. Robust scaling (median + IQR) is less sensitive to outliers than standard scaling (mean + std).
- The pipeline ensures **no data leakage**: scaling parameters are learned **inside** each CV fold.
- Alternatives to try later:
  - **Tree ensembles** (e.g. Random Forests) for non-linear baselines.
  - **Elastic Net** for embedded sparsity (tuning needed).

### Guarding against leakage in CV

By wrapping preprocessing and the estimator in a **single pipeline** and passing it to `cross_val_score`, we guarantee that:
- **Scaling** and **model fitting** occur **only on the training folds**.
- The held-out fold remains unseen until scoring, yielding **honest** performance estimates.


In [ ]:
# CV evaluation helper
def evaluate_cv(x_df: pd.DataFrame, y_np: np.ndarray, estimator=ESTIMATOR, cv=CV, scorer=SCORER) -> tuple[float, float]:
    """
    Returns (mean_bal_acc, std_bal_acc) from K-fold CV.
    """
    scores = cross_val_score(estimator, x_df, y_np, cv=cv, scoring=scorer, n_jobs=None)
    return float(scores.mean()), float(scores.std())

### Run the sweep over noise counts, collecting balanced accuracy

In [ ]:
def run_sweep(noise_counts, noise_dist, noise_distribution_params) -> pd.DataFrame:
    results = []
    for k_noise in noise_counts:
        # Build a dataset with a fixed informative features, samples, class separation and seed but variable
        # number of pure-noise features
        fixed_cfg = DatasetConfig(
            n_noise=k_noise,
            noise_distribution=noise_dist,
            noise_distribution_params=noise_distribution_params,
            n_informative=8,
            class_configs=[
                ClassConfig(n_samples=15),
                ClassConfig(n_samples=15),
            ],
            class_sep=[1.2],
            random_state=42,
        )
        x_k, y_k, meta_k = generate_dataset(fixed_cfg, return_dataframe=True)
        mean_ba, std_ba = evaluate_cv(x_k, y_k)
        results.append(
            {
                "dist": str(noise_dist),
                "scale": noise_distribution_params,
                "n_noise": k_noise,
                "n_informative": len(meta_k.informative_idx),
                "bal_acc_mean": mean_ba,
                "bal_acc_std": std_ba,
            }
        )
    return pd.DataFrame(results).sort_values("n_noise").reset_index(drop=True)


result_df = run_sweep(NOISE_COUNTS, "normal", {"loc": 0, "scale": 1.0})

### What trends to expect

**Balanced accuracy** typically **decreases** as `n_noise` increases (harder signal recovery).

### Plot results

In [ ]:
# Balanced accuracy vs number of noise features
plt.figure()
plt.plot(result_df["n_noise"], result_df["bal_acc_mean"], marker="o")
plt.title("Balanced accuracy vs number of noise features")
plt.xlabel("Number of normal distributed noise features")
plt.ylabel("Balanced accuracy (CV mean)")
plt.show()

### Reading the plots carefully

- Expect **non-monotonic fluctuations** due to finite-sample randomness—don’t over-fit to a single curve.
- Look for the **overall slope**: does performance degrade roughly as dimensionality rises?
- If a method looks unusually stable, verify you are **not leaking information** and that regularization is properly tuned.

### Computational notes

- This sweep is lightweight (single model, 5-fold CV).
- As you add methods (e.g., nested CV, repeated subsampling, multiple models), runtime will grow quickly.
- Keep random seeds fixed for **comparability**, and cache intermediate results when exploring many settings.

### Interpretation checklist

- **Performance drop**: As `n_noise` grows, balanced accuracy should typically **decrease** unless regularization and sample size are strong.
- **Sample size matters**: With fixed effect size, increasing `n_samples` generally improves both performance and discovery metrics.
- **Regularization**: Linear models with proper regularization can resist some noise, but the *curse of dimensionality* still applies.

### Biomedical context: why this matters

Omics datasets (e.g., gene expression, proteomics, methylation) often have **p ≫ n**:
- Thousands of features with **few dozens of samples**.
- Many features are unrelated to the phenotype—i.e., **irrelevant** for prediction.

Understanding how irrelevant features affect **generalization** and **feature discovery** helps you:
- Choose appropriate **regularization** and **model families**,
- Design **feature selection** protocols that avoid false discoveries,
- Plan for **validation** on independent cohorts.


# Step 3 — Comparing noise distributions

Repeat the sweep with:

1. `noise_distribution="uniform", noise_distribution_params={"low": -2, "high": 2}`
 >**Uniform** noise (bounded support) usually yields tighter extremes than normal at small `n`, reducing outlier risks.

2. `noise_distribution="laplace", noise_distribution_params={"loc": 0, "scale": 1.0}`
>**Laplace** noise (heavier tails) increases the chance of **extreme values**, which can inflate spurious correlations and destabilize rankings.

When sweeping distributions, keep **`n_samples`, `n_informative`, and `class_sep`** fixed to isolate the distributional effect. Then compare curves (balanced accuracy) across distributions.

In [ ]:
df_uniform = run_sweep(NOISE_COUNTS, "uniform", {"low": -2, "high": 2})
df_laplace = run_sweep(NOISE_COUNTS, "laplace", {"loc": 0, "scale": 1.0})
display(df_uniform, df_laplace)

In [ ]:
# Balanced accuracy vs number of uniform noise features
plt.figure()
plt.plot(df_uniform["n_noise"], df_uniform["bal_acc_mean"], marker="o")
plt.title("Balanced accuracy vs number of noise features")
plt.xlabel("Number of noise features")
plt.ylabel("Balanced accuracy (CV mean)")
plt.show()

In [ ]:
# Balanced accuracy vs number of laplace noise features
plt.figure()
plt.plot(df_laplace["n_noise"], df_laplace["bal_acc_mean"], marker="o")
plt.title("Balanced accuracy vs number of noise features")
plt.xlabel("Number of noise features")
plt.ylabel("Balanced accuracy (CV mean)")
plt.show()

## Reflection
**Model performance** (if you ran the optional classifier):
  - How much did adding noise affect classification accuracy?
  - What might happen if you add *hundreds* of noise features?

**Think ahead**:
In real biomedical studies, many measured variables are irrelevant.
Why is it important to detect and filter them before applying complex models?

# Step 4 Spurious correlations

Irrelevant features should be **uncorrelated** with `y` (up to sampling noise) and show no systematic correlation with informative features.

We’ll compute:
- Absolute correlation between each **noise** feature and the **label** `y`.

In [ ]:
# Work on the first 20 features of the initial noise dataset (xn, yn, meta_n)
y_numeric = pd.Series(yn, name="y").astype(float)

# 1) Correlation of the first 20 features with the label
y_corr = xn.iloc[:, :20].apply(lambda s: s.corr(y_numeric)).abs().sort_values(ascending=False)

print("Top 10 of the first 20 features (8 informative, 12 noise)\n|abs(corr(features, y))|:")
display(y_corr.to_frame().head(10).round(3))

In [ ]:
# Work on the full initial noise dataset (xn, yn, meta_n)
y_numeric = pd.Series(yn, name="y").astype(float)
print(f"Number of features: {xn.shape[1]}, Number of samples: {xn.shape[0]}\n")

# 1) Correlation of features with the label
y_corr = xn.apply(lambda s: s.corr(y_numeric)).abs().sort_values(ascending=False)

print("Top 10 |abs(corr(features, y))|:")
display(y_corr.to_frame().head(10).round(3))

### Spurious correlations grow with dimensionality

Even if each noise feature is uninformative, the **largest absolute correlation** among many noise variables can be non-trivial **by chance**.

Rule of thumb (for intuition): the maximum spurious correlation scales roughly like
$$
\max_j \lvert r_{j,y}\rvert \sim \sqrt{\tfrac{2\log p}{n}}
$$
where \(p\) is the number of features and \(n\) the sample size.


➡️ As you add more noise features or reduce sample size, **chance correlations** become more prominent.


## Quick Takeaway
- **Irrelevant features** add no predictive value and can **dilute** signal. Synthetic data allows us to control how much noise exists, so we can test how methods behave in high-noise vs. low-noise scenarios.
- As the number of irrelevant features increases, model performance (e.g., balanced accuracy) typically **decreases** due to the *curse of dimensionality*.
- Different noise distributions (normal, uniform, Laplace) have distinct characteristics that can affect model training and evaluation.
- Spurious correlations between noise features and the label can arise by chance, especially as dimensionality increases.
- In high-noise settings, feature selection (see MC XXX) becomes crucial.

## Next Steps

In **Lesson 3c: Correlated Features**, you will:
- Simulate features that **move together** (e.g., biological pathways).
- Visualize correlation structures.
- See how **redundancy** complicates analysis.

➡️ Continue with: **[`03c_correlated_features.ipynb`](03c_correlated_features.ipynb)`**
